## Explore tome and train models

A tome consists of four things

- **The keysets** - A simple list of keys that are included in this tome.
- **The dataframes** - The data.
- **The manifest** - A listing of availble keyset/dataframe files and other metadata about the tome.
- **The header** - This provides the target list of keys while making the tome. Generally used by internal processes only. This is what makes tomes "immutable" once started or finished.

Because they can be large, the combined dataframe data are saved in separate files called "pages". You can set a max (memory) size for each page when making a tome.

Let's read in that tome and train a model to go from number of footsteps to rank, obviously.

_**Run this notebook as-is.**_

In [ ]:
from pureskillgg_makenew_pyskill.notebook import setup_notebook

In [ ]:
setup_notebook(silent=True)

In [ ]:
# %load ../usual_suspects.py
# pylint: disable=unused-import
import time
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pureskillgg_dsdk.tome import create_tome_curator

pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 150)
pd.set_option("display.min_rows", 150)
# pd.set_option('display.float_format', '{:.4f}'.format)

curator = create_tome_curator()


In [ ]:
df = curator.get_dataframe('footsteps_by_rank')
keyset = curator.get_keyset('footsteps_by_rank')

In [ ]:
print(len(df),len(keyset))

In [ ]:
df.head(25)

## Training a model left as exercise to user

:) 

In [ ]:
df['rank_fixed']=df['rank'].apply(lambda x: x/2).apply(round)

df.groupby('rank_fixed',as_index=False).max()